In [1]:
data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "schoffelen2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

In [2]:
# from utils.compression import compress_directories, decompress_directories

# for base_path, batch_type in data_partition.items():
#     decompress_directories(
#         source_path=f'downloaded_data/{base_path}',
#         destination_path=f'data/{base_path}',
#         checksum_file_name="checksums.txt",
#         delete_compressed_files=True,
#         num_workers=26
#     )

In [3]:
from config import SimpleConvConfig
from models.simpleconv import SimpleConv
import torch

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=128,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=False,
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=None,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

In [4]:
from train.training_session_v0 import TrainingSessionV0
from config import TrainingConfigV0
import multiprocessing

config = TrainingConfigV0(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Pre-processing parameters
    # Brain
    new_freq=100,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=20,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Hyperparameters
    learning_rate=3e-4,
    weight_decay=1e-4,
    epochs=50,
    batch_size=256,
    use_clip_loss=True,
    use_mse_loss=True,
    alpha=0.6,
    random_test_size=10,
    seed=42,
)

session = TrainingSessionV0(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="data",
    save_path="saves/phase1/objective/Dec18_CLIP_MSE_384",
    clear_cache=False,
)

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

SimpleConv initialized with 8448160 parameters, cond: ['study', 'subject']
Merger False, merger channels 0
ConvBlocks: 6, hidden_dim: 384, params 7973376


In [5]:
try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=multiprocessing.cpu_count() - 2,
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower than expected.


2024-12-19 19:02:47,197	INFO worker.py:1821 -- Started a local Ray instance.
Training Epoch 1: 100%|██████████| 135/135 [03:36<00:00,  1.60s/it]


Testing at epoch 1
Test unseen_subject completed. Accuracy: 0.0350, Top 1: 0.0536, Top 5: 0.2055, Top 10: 0.3267, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.0348, Top 1: 0.0516, Top 5: 0.1838, Top 10: 0.3020, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.0150, Top 1: 0.0286, Top 5: 0.1471, Top 10: 0.2741, Perplexity: 0.0000
Epoch 1 completed in 3.93m. 0.03m per recording.


Training Epoch 2: 100%|██████████| 135/135 [03:35<00:00,  1.60s/it]


Testing at epoch 2
Test unseen_subject completed. Accuracy: 0.0792, Top 1: 0.1105, Top 5: 0.2885, Top 10: 0.4192, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.0597, Top 1: 0.0854, Top 5: 0.2737, Top 10: 0.4073, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.0512, Top 1: 0.0698, Top 5: 0.2667, Top 10: 0.3836, Perplexity: 0.0000
Epoch 2 completed in 3.90m. 0.03m per recording.


...

Testing at epoch 20
Test unseen_subject completed. Accuracy: 0.1677, Top 1: 0.2432, Top 5: 0.5074, Top 10: 0.6115, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.2222, Top 1: 0.2934, Top 5: 0.5937, Top 10: 0.7192, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.1470, Top 1: 0.2139, Top 5: 0.5042, Top 10: 0.5927, Perplexity: 0.0000
Epoch 20 completed in 3.89m. 0.03m per recording.


Training Epoch 21: 100%|██████████| 135/135 [03:32<00:00,  1.58s/it]


Testing at epoch 21
Test unseen_subject completed. Accuracy: 0.1765, Top 1: 0.2385, Top 5: 0.4755, Top 10: 0.5895, Perplexity: 0.0000
Test unseen_task completed. Accuracy: 0.1978, Top 1: 0.2633, Top 5: 0.5400, Top 10: 0.6611, Perplexity: 0.0000
Test unseen_both completed. Accuracy: 0.1307, Top 1: 0.1879, Top 5: 0.4084, Top 10: 0.5343, Perplexity: 0.0000
Epoch 21 completed in 3.87m. 0.03m per recording.


Training Epoch 22:   0%|          | 0/135 [00:02<?, ?it/s]

Exited


In [ ]:
# import os

# def delete_file_recursive(directory, filename):
#     for root, dirs, files in os.walk(directory):
#         if filename in files:
#             file_path = os.path.join(root, filename)
#             try:
#                 os.remove(file_path)
#                 print(f"Deleted: {file_path}")
#             except PermissionError:
#                 print(f"Permission denied: {file_path}")
#             except Exception as e:
#                 print(f"Error deleting {file_path}: {e}")

# directory = os.getcwd()
# filename = 'training_log.log'
# delete_file_recursive(directory, filename)